In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from captum.attr import IntegratedGradients, GuidedGradCam

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [3]:
torch.manual_seed(123)
np.random.seed(123)

In [4]:
input = torch.randn(2, 3, 32, 32, requires_grad=True)
baseline = torch.zeros(2, 3, 32, 32, requires_grad=True)

In [50]:
net.forward(input).shape

torch.Size([2, 10])

In [5]:

ig = IntegratedGradients(net)
attributions, delta = ig.attribute(input, baseline, target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)


IG Attributions: tensor([[[[ 0.0000e+00, -2.8690e-06,  6.6988e-06,  ...,  9.4744e-06,
            8.8631e-06, -7.9254e-05],
          [ 5.5141e-05, -1.2744e-05,  3.4356e-04,  ...,  1.0261e-04,
            1.5972e-05,  7.9453e-06],
          [-3.4709e-05, -1.7314e-05,  9.7482e-06,  ...,  1.6306e-04,
            4.2891e-06,  9.1493e-05],
          ...,
          [-3.4535e-05,  1.9842e-05,  3.3865e-05,  ..., -4.0647e-05,
            3.4614e-05,  3.6084e-05],
          [-5.9232e-06, -1.8482e-05, -1.8358e-06,  ...,  4.0610e-05,
            1.5057e-05,  8.7806e-06],
          [-2.5942e-06,  3.0662e-06, -3.7107e-06,  ..., -2.0345e-05,
           -1.5147e-05,  1.1577e-07]],

         [[ 0.0000e+00, -4.3485e-06, -4.9521e-05,  ..., -1.5254e-06,
           -1.8721e-05,  1.4769e-05],
          [ 3.0744e-05,  1.4965e-05,  9.1396e-05,  ...,  1.4488e-04,
           -2.0859e-06, -1.4171e-05],
          [-3.9065e-06,  2.2861e-05,  7.2429e-05,  ..., -7.2163e-05,
            6.9770e-05, -8.7776e-06],
   

In [6]:
GC = GuidedGradCam(net, net.conv2)
attributions = GC.attribute(input, target=0) # no need to give target for single output
print('GC Attributions:', attributions)

GC Attributions: tensor([[[[ 0.0000e+00, -1.1117e-08, -2.9576e-08,  ...,  3.0378e-08,
           -6.3717e-09,  2.8233e-08],
          [-2.4147e-08, -3.9486e-08, -1.3639e-07,  ..., -6.7290e-08,
            2.9220e-08,  2.3149e-08],
          [-7.9752e-08, -4.9009e-08, -4.1587e-08,  ..., -1.1897e-07,
           -1.4342e-08, -3.7941e-08],
          ...,
          [ 7.0228e-09, -9.6896e-09,  1.8578e-08,  ...,  1.9460e-08,
           -2.4690e-08, -2.1526e-08],
          [ 1.0502e-08,  3.0092e-10, -5.3806e-10,  ..., -7.4588e-08,
           -8.0859e-09,  3.3365e-09],
          [ 1.3143e-09, -7.9230e-09,  9.5321e-09,  ...,  2.0429e-08,
            3.1880e-08,  6.6201e-09]],

         [[ 0.0000e+00, -1.2688e-08, -6.4509e-08,  ...,  1.5590e-08,
            1.7910e-08,  1.6425e-08],
          [ 1.7929e-08,  5.7960e-09,  9.6442e-08,  ...,  2.2237e-08,
            4.5785e-08, -1.6369e-08],
          [-2.1238e-08,  1.5301e-08, -1.5054e-07,  ..., -1.6615e-08,
           -6.7800e-08, -6.1740e-10],
   

/volatile/jl274628/Runs/venv_local/lib/python3.7/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:65: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."
